In [ ]:
%pip install openai == 0.28
%pip install tiktoken
%pip install tqdm
%pip install matplotlib
%pip install sympy

In [3]:
import numpy as np
import openai
import tiktoken
from tqdm.auto import trange, tqdm
import time
import os
import json
from tqdm import tqdm
import re
from types import NoneType
import multiprocessing.dummy
from io import StringIO
from contextlib import redirect_stdout
import signal
from contextlib import contextmanager
import matplotlib.pyplot as plt
import sys
import ast
import copy

In [4]:
import pandas as pd
from IPython.display import display_latex
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm


## LLM SETUP OPEN AI

In [ ]:
OPENAI_API_KEY = "YOUR API KEY"

In [ ]:
# Code generating prompts
system_prompt = """
You are a helpful assistant, assinged with generating docstrings to python functions and classes. 
You have vast knowledge of python coding and libraries.
You know the PEP8 coding conventions.
"""
my_prompt = """
Provide a good docstring to the following code: {code}
Correct format example:
code = 

def multiply_and_sum(lst):
    res = 0
    for i, item in enumerate(lst):
        res += item * i 
    return res

solution =

    Iteratively computes the sum of all elements in a list of integers after multiplying each element by its index in the list.
    Args:
    - lst (list): a list of integers
    Returns:
    - int: the sum of all elements in the list after preforming the multipication of the elements by their indices.
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_prompt),
])

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-1106-preview")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain_solution = prompt | llm | output_parser

In [ ]:
def generate_docstring_llm(code, docstring_col, chain_solution=chain_solution):
    llm_docstring = chain_solution.invoke({
        "code": code
    })
    return docstring_col, llm_docstring

In [ ]:
data = pd.read_csv("Benchmark database code chunks.csv")

In [ ]:
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm

tqdm.pandas()

with get_openai_callback() as cb:
    new_docstring_col = 'new_docstring' # change col name for each model
    data[new_docstring_col] = None
    missing_values_mask = data[new_docstring_col].isna()
    print(f"Generating docstrings by {new_docstring_col}...")
    
    data.loc[missing_values_mask, [new_docstring_col]] = (
        data.loc[missing_values_mask, 'Function'].progress_apply(
            lambda x: generate_docstring_llm(x, new_docstring_col)[1]
        )
    )
    
    # Debugging: Print the generated docstrings
    print(data.loc[missing_values_mask, [new_docstring_col]])

print(cb)
